In [ ]:
#Run every cell, in order, until instructed otherwise

In [63]:
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
model=LogisticRegression()
import itertools

In [65]:
ufc_fights= pd.read_csv('ufc_fights.csv')
ufc_fights_crap= pd.read_csv('ufc_fights_crap.csv')
ufcfightscrap= pd.read_csv('ufc_fights_crap.csv')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

In [ ]:
def subsets(n):
    lists=[]
    for i in range(1,n+1):
        lists.append(i)
    subsets=[[]]
    n=len(lists)
    for i in range(1,n+1):
        data = itertools.combinations(lists,i)
        subsets.extend(list(data))
    return subsets

In [66]:
#adding mask to get rid of unknown stance
stance_mask_a=ufc_fights['fighter_stance']!=5
stance_mask_b=ufc_fights['opponent_stance']!=5
ufc_fights_stance=ufc_fights[stance_mask_a&stance_mask_b]

#gets rid of all unknown, NaN in ufc_fights_stance
mask=ufc_fights_stance['fighter_wins'] != 'unknown'

for col in ufc_fights_stance.columns:
    mask1=ufc_fights_stance[col] != 'unknown'
    mask2=ufc_fights_stance[col] != 'NaN'
    mask3=ufc_fights_stance[col] != '--'
    mask=mask&mask1&mask2&mask3

#gets rid of ufc debut in ufc_fights_stance
maska=ufc_fights_stance['fighter_inf_sig_strikes_attempts_avg'] != 0
maskb=ufc_fights_stance['opponent_inf_sig_strikes_attempts_avg'] != 0
mask=mask&maska&maskb

#applies all masks resulting in ufc_fights_clean
ufc_fights_clean=ufc_fights_stance[mask]

In [53]:
#here we list all of the possible statistics. Next lets identify based on intuition which could help predict method
record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

In [67]:
#Getting rid of all fights whose method is DQ, Overturned, or CNC or any draws
method_mask1=ufc_fights_clean['method']!='DQ'
method_mask2=ufc_fights_clean['method']!='Overturned'
method_mask3=ufc_fights_clean['method']!='CNC'
method_mask4=ufc_fights_clean['result']!='D'
method_mask=method_mask1&method_mask2&method_mask3&method_mask4
ufc_fights_clean=ufc_fights_clean[method_mask]
method_mask.sum()

3620

In [55]:
#Now lets get rid of the different types of decisions (all become DEC)
ufc_fights_clean['method'] = ufc_fights_clean['method'].replace(['U-DEC'],'DEC')
ufc_fights_clean['method'] = ufc_fights_clean['method'].replace(['S-DEC'],'DEC')
ufc_fights_clean['method'] = ufc_fights_clean['method'].replace(['M-DEC'],'DEC')

In [68]:
#here we list all of the possible statistics. Next lets identify based on intuition which could help predict method
mlp_stats_grouped=[[u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses'], 
                   [u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'opponent_ko_wins', 
                   u'opponent_ko_losses'],
                   [u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses'],
                   [u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses'],
                   [u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses'],
                    [u'fighter_age', 
                     u'opponent_age'],
                    [u'fighter_height',
                     u'fighter_reach', 
                    u'opponent_height',
                     u'opponent_reach'],
                    [u'fighter_inf_knockdowns_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_abs_knockdowns_avg'],
                    [u'fighter_inf_pass_avg',
                    u'fighter_abs_pass_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_abs_pass_avg'],
                    [u'fighter_inf_reversals_avg',
                    u'fighter_abs_reversals_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_abs_reversals_avg'],
                    [u'fighter_inf_sub_attempts_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_abs_sub_attempts_avg'],
                    [u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg'],
                    [u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg'],
                    [u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg'],
                    [u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg'],
                    [u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg'],
                    [u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg'],
                    [u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg'],
                    [u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg'],
                    [u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']
                    ]
len(mlp_stats_grouped)

20

In [69]:
mlp_stats=[]
for stat in mlp_stats_grouped:
    mlp_stats.extend(stat)

In [45]:
#taking all stats gives us under .505 (this is bad...)
mlp=MLPClassifier(max_iter=1000,alpha=.00001,solver='sgd',random_state=3)
Xo=ufc_fights_clean[mlp_stats].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=5).mean()

0.5059780338020753

In [40]:
mlp_single_scores=[]
mlp=MLPClassifier(max_iter=1000,alpha=.0001,solver='sgd',random_state=3)
for stat in mlp_stats_grouped:
    Xo=ufc_fights_clean[stat].iloc[0:2091]
    yo=ufc_fights_clean.iloc[0:2091]['method']
    mlp_single_scores.append([stat,cross_val_score(mlp,Xo,yo,cv=5).mean()])
    print([stat,cross_val_score(mlp,Xo,yo,cv=5).mean()])

[[u'fighter_L5Y_wins', u'fighter_L5Y_losses', u'fighter_L2Y_wins', u'fighter_L2Y_losses', u'opponent_L5Y_wins', u'opponent_L5Y_losses', u'opponent_L2Y_wins', u'opponent_L2Y_losses'], 0.49449700382891776]
[[u'fighter_ko_wins', u'fighter_ko_losses', u'opponent_ko_wins', u'opponent_ko_losses'], 0.5829696147616513]
[[u'fighter_L5Y_ko_wins', u'fighter_L5Y_ko_losses', u'fighter_L2Y_ko_wins', u'fighter_L2Y_ko_losses', u'opponent_L5Y_ko_wins', u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins', u'opponent_L2Y_ko_losses'], 0.5179432587581487]
[[u'fighter_sub_wins', u'fighter_sub_losses', u'opponent_sub_wins', u'opponent_sub_losses'], 0.5293876593236151]
[[u'fighter_L5Y_sub_wins', u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins', u'fighter_L2Y_sub_losses', u'opponent_L5Y_sub_wins', u'opponent_L5Y_sub_losses', u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses'], 0.49593814208776177]
[[u'fighter_age', u'opponent_age'], 0.5007182797914413]
[[u'fighter_height', u'fighter_reach', u'opponent_heigh

In [43]:
[score[1] for score in mlp_single_scores if score[1]>.502]

[0.5829696147616513,
 0.5179432587581487,
 0.5293876593236151,
 0.5045574995850575,
 0.5026344556812783]

In [58]:
mlp_base_set=[]
for i in (1,2,3,14,18):
    mlp_base_set.extend(mlp_stats_grouped[i])

In [59]:
mlp_base_set_grouped=[]
for i in (1,2,3,14,18):
    mlp_base_set_grouped.append(mlp_stats_grouped[i])

In [60]:
mlp_add_backs=[]
for i in (0,4,5,6,7,8,9,10,11,12,13,15,16,17,19):
    mlp_add_backs.append(mlp_stats_grouped[i]) 

In [61]:
mlp=MLPClassifier(max_iter=2000,alpha=.001,solver='sgd',random_state=3)

In [62]:
#COOL... WE ALMOST IMMEDIATELY GOT A HUGE IMPROVEMENT
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=5).mean()

0.6427453741064428

In [56]:
time cross_val_score(mlp,Xo,yo,cv=4).mean()

CPU times: user 1min 3s, sys: 367 ms, total: 1min 3s
Wall time: 32.6 s


0.6307901657839021

In [49]:
#now lets do some add backs

In [50]:
len(mlp_add_backs)

15

In [ ]:
#clearly cant do all 2^15 (would take years) so lets see whats reasonable. It takes 33 seconds to do the simplest
#computation with cv=4. If we go up to subsets of size 2, it will take 120*33/60/60=1.1 hours

In [ ]:
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
cross_val_score(mlp,Xo,yo,cv=5).mean()

In [ ]:
#this adds subsets of size 1 from mlp_add_backs to mlp_base_set
mlp=MLPClassifier(max_iter=2000,alpha=.001,solver='sgd',random_state=3)
yo=ufc_fights_clean.iloc[0:2091]['method']
mlp_data_points=[]
SUB=[S for S in subsets(15) if len(S)<2]
for S in SUB:
    candidate=[]
    candidate.extend(mlp_base_set)
    for i in S:
        candidate.extend(mlp_add_backs[i-1])
    Xo=ufc_fights_clean[candidate].iloc[0:2091]
    point=[S,cross_val_score(mlp,Xo,yo,cv=4).mean()]
    mlp_data_points.append(point)
    print(point)

[[], 0.6307901657839021]
[(1,), 0.6135725954740922]
[(2,), 0.650402372109038]
[(3,), 0.6001745382885357]
[(4,), 0.5011959444114782]
[(5,), 0.632225115931518]
[(6,), 0.6346170047544742]
[(7,), 0.6331902961839666]
[(8,), 0.6355730276990249]
[(9,), 0.6250540281165982]
[(10,), 0.610225599437375]
[(11,), 0.603050848699296]
[(12,), 0.6303185644271554]
[(13,), 0.6240980051720475]
[(14,), 0.6121468026343744]


In [18]:
its=[8,4,3,9,2]

In [19]:
its.remove(4)

In [20]:
its

[8, 3, 9, 2]

In [21]:
mlp_add_backs[1]

[u'fighter_L5Y_sub_wins',
 u'fighter_L5Y_sub_losses',
 u'fighter_L2Y_sub_wins',
 u'fighter_L2Y_sub_losses',
 u'opponent_L5Y_sub_wins',
 u'opponent_L5Y_sub_losses',
 u'opponent_L2Y_sub_wins',
 u'opponent_L2Y_sub_losses']

In [14]:
#the highest is 2 (actually the index 1 because of the stupid way I wrote it) with .65 so we add that
#removing from mlp_add_backs and adding to mlp_base_set
mlp_base_set.extend(mlp_add_backs[1])
mlp_add_backs.remove(mlp_add_backs[1])

In [15]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=5).mean()

0.6695352071564705

In [53]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=6).mean()

0.6532651492802782

In [54]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=7).mean()

0.6585091967117063

In [55]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=8).mean()

0.648520443976485

In [56]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=9).mean()

0.6656619366031827

In [57]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=10).mean()

0.6632227802708353

In [58]:
#COOL... WOWWW THIS IS GETTING GOOD. ALMOST AT OUR GOAL
Xo=ufc_fights_clean[mlp_base_set].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=11).mean()

0.6647780207300736

In [ ]:
#we tried another round of adding one or two element sets to mlp_base_set to no avail. Now lets try refining
#mlp_base_set

In [17]:
mlp_base_set_grouped_refined=[
    [u'fighter_ko_wins',
    u'opponent_ko_wins'],
  [u'fighter_ko_losses',
  u'opponent_ko_losses'],
 [u'fighter_L5Y_ko_wins',
  u'opponent_L5Y_ko_wins'],
  [u'fighter_L5Y_ko_losses',
  u'opponent_L5Y_ko_losses'],
  [u'fighter_L2Y_ko_wins',
   u'opponent_L2Y_ko_wins'],
  [u'fighter_L2Y_ko_losses',
    u'opponent_L2Y_ko_losses'],
 [u'fighter_sub_wins',
  u'opponent_sub_wins'],
  [u'fighter_sub_losses',
  u'opponent_sub_losses'],
 [u'fighter_inf_head_strikes_landed_avg',
  u'opponent_inf_head_strikes_landed_avg'],
  [u'fighter_abs_head_strikes_landed_avg',
  u'opponent_abs_head_strikes_landed_avg'],
  [u'fighter_inf_head_strikes_attempts_avg',
   u'opponent_inf_head_strikes_attempts_avg'],
  [u'fighter_abs_head_strikes_attempts_avg',
  u'opponent_abs_head_strikes_attempts_avg'],
 [u'fighter_inf_clinch_strikes_landed_avg',
  u'opponent_inf_clinch_strikes_landed_avg'],
  [u'fighter_abs_clinch_strikes_landed_avg',
  u'opponent_abs_clinch_strikes_landed_avg'],
  [u'fighter_inf_clinch_strikes_attempts_avg',
   u'opponent_inf_clinch_strikes_attempts_avg'],
  [u'fighter_abs_clinch_strikes_attempts_avg',
  u'opponent_abs_clinch_strikes_attempts_avg'],
  [u'fighter_L5Y_sub_wins',
   u'opponent_L5Y_sub_wins'],
  [u'fighter_L5Y_sub_losses',
   u'opponent_L5Y_sub_losses'],
  [u'fighter_L2Y_sub_wins',
     u'opponent_L2Y_sub_wins'],
  [u'fighter_L2Y_sub_losses',
  u'opponent_L2Y_sub_losses']]

In [18]:
len(mlp_base_set_grouped_refined)

20

In [19]:
test_scores=[]
test=[]
for j in range(len(mlp_base_set_grouped_refined)):
    test.extend(mlp_base_set_grouped_refined[j])
Xo=ufc_fights_clean[test].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
test_scores.append(['empty',cross_val_score(mlp,Xo,yo,cv=5).mean()])
for i in range(len(mlp_base_set_grouped_refined)):
    test=[]
    for j in range(len(mlp_base_set_grouped_refined)):
        if j!=i:
            test.extend(mlp_base_set_grouped_refined[j])
    Xo=ufc_fights_clean[test].iloc[0:2091]
    yo=ufc_fights_clean.iloc[0:2091]['method']
    test_scores.append([i,cross_val_score(mlp,Xo,yo,cv=5).mean()])
    print([i,cross_val_score(mlp,Xo,yo,cv=5).mean()])

[0, 0.5973031598587478]


/opt/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[1, 0.5925390475209331]
[2, 0.6446294992645502]
[3, 0.6355305254603]
[4, 0.6522827561339949]
[5, 0.6570583151617131]
[6, 0.612100295896934]
[7, 0.5911070666140119]
[8, 0.6599451703553625]
[9, 0.6412904998197148]
[10, 0.6508439072131317]
[11, 0.6451251409373695]
[12, 0.6408108835127602]
[13, 0.6431929396816675]
[14, 0.6513212341821053]
[15, 0.6484824550860504]
[16, 0.6360192991191772]
[17, 0.6494233729961139]
[18, 0.6494119263062104]
[19, 0.641301946509618]


In [20]:
#Why is this smaller than our .669?
test_scores[0]

['empty', 0.658529214814306]

In [21]:
test=[]
for j in range(len(mlp_base_set_grouped_refined)):
    test.extend(mlp_base_set_grouped_refined[j])

In [23]:
set(test)==set(mlp_base_set)

True

In [24]:
#it seems that the order matters quite a lot...
test==mlp_base_set

False

In [26]:
mlp_base_set_grouped_refined[8]

[u'fighter_inf_head_strikes_landed_avg',
 u'opponent_inf_head_strikes_landed_avg']

In [27]:
#removing 8 seems to be higher by a little bit. Maybe we'll reach 67? 
test=[]
for j in range(len(mlp_base_set_grouped_refined)):
    if j!=8:
        test.extend(mlp_base_set_grouped_refined[j])
Xo=ufc_fights_clean[test].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=6).mean()

0.6465285184214511

In [29]:
#here I am preserving the original order which seems better... and getting rid of inf_head_strikes_landed
test_prime=[u'fighter_ko_wins',
 u'fighter_ko_losses',
 u'opponent_ko_wins',
 u'opponent_ko_losses',
 u'fighter_L5Y_ko_wins',
 u'fighter_L5Y_ko_losses',
 u'fighter_L2Y_ko_wins',
 u'fighter_L2Y_ko_losses',
 u'opponent_L5Y_ko_wins',
 u'opponent_L5Y_ko_losses',
 u'opponent_L2Y_ko_wins',
 u'opponent_L2Y_ko_losses',
 u'fighter_sub_wins',
 u'fighter_sub_losses',
 u'opponent_sub_wins',
 u'opponent_sub_losses',
 u'fighter_inf_head_strikes_attempts_avg',
 u'fighter_abs_head_strikes_landed_avg',
 u'fighter_abs_head_strikes_attempts_avg',
 u'opponent_inf_head_strikes_attempts_avg',
 u'opponent_abs_head_strikes_landed_avg',
 u'opponent_abs_head_strikes_attempts_avg',
 u'fighter_inf_clinch_strikes_landed_avg',
 u'fighter_inf_clinch_strikes_attempts_avg',
 u'fighter_abs_clinch_strikes_landed_avg',
 u'fighter_abs_clinch_strikes_attempts_avg',
 u'opponent_inf_clinch_strikes_landed_avg',
 u'opponent_inf_clinch_strikes_attempts_avg',
 u'opponent_abs_clinch_strikes_landed_avg',
 u'opponent_abs_clinch_strikes_attempts_avg',
 u'fighter_L5Y_sub_wins',
 u'fighter_L5Y_sub_losses',
 u'fighter_L2Y_sub_wins',
 u'fighter_L2Y_sub_losses',
 u'opponent_L5Y_sub_wins',
 u'opponent_L5Y_sub_losses',
 u'opponent_L2Y_sub_wins',
 u'opponent_L2Y_sub_losses']

In [30]:
Xo=ufc_fights_clean[test_prime].iloc[0:2091]
yo=ufc_fights_clean.iloc[0:2091]['method']
cross_val_score(mlp,Xo,yo,cv=5).mean()

0.6465433858164065

In [ ]:
#ok so mlp_base_set seems pretty solid.

In [ ]:
#Now that we tried to delete some things, lets try picking single things by hand from add_backs

In [35]:
mlp_add_backs_refined=[
 [u'fighter_age', u'opponent_age'],
 [u'fighter_inf_knockdowns_avg',
  u'opponent_inf_knockdowns_avg'],
  [u'fighter_abs_knockdowns_avg',
  u'opponent_abs_knockdowns_avg'],
 [u'fighter_inf_pass_avg',
  u'fighter_abs_pass_avg'],
  [u'opponent_inf_pass_avg',
  u'opponent_abs_pass_avg'],
 [u'fighter_inf_reversals_avg',
  u'opponent_inf_reversals_avg'],
  [u'fighter_abs_reversals_avg',
  u'opponent_abs_reversals_avg'],
 [u'fighter_inf_sub_attempts_avg',
  u'opponent_inf_sub_attempts_avg'],
  [u'fighter_abs_sub_attempts_avg',
  u'opponent_abs_sub_attempts_avg'],
 [u'fighter_inf_takedowns_landed_avg',
  u'opponent_inf_takedowns_landed_avg'],
  [u'fighter_abs_takedowns_landed_avg',
  u'opponent_abs_takedowns_landed_avg'],
  [u'fighter_inf_takedowns_attempts_avg',
   u'opponent_inf_takedowns_attempts_avg'],
  [u'fighter_abs_takedowns_attempts_avg',
  u'opponent_abs_takedowns_attempts_avg'],
 [u'fighter_inf_sig_strikes_landed_avg',
  u'opponent_inf_sig_strikes_landed_avg'],
  [u'fighter_abs_sig_strikes_landed_avg',
  u'opponent_abs_sig_strikes_landed_avg'],
  [u'fighter_inf_sig_strikes_attempts_avg',
   u'opponent_inf_sig_strikes_attempts_avg'],
  [u'fighter_abs_sig_strikes_attempts_avg',
  u'opponent_abs_sig_strikes_attempts_avg'],
 [u'fighter_inf_total_strikes_landed_avg',
  u'opponent_inf_total_strikes_landed_avg'],
  [u'fighter_abs_total_strikes_landed_avg',
  u'opponent_abs_total_strikes_landed_avg'],
  [u'fighter_inf_total_strikes_attempts_avg',
   u'opponent_inf_total_strikes_attempts_avg'],
  [u'fighter_abs_total_strikes_attempts_avg',
  u'opponent_abs_total_strikes_attempts_avg'],
 [u'fighter_inf_body_strikes_landed_avg',
  u'opponent_inf_body_strikes_landed_avg'],
  [u'fighter_abs_body_strikes_landed_avg',
  u'opponent_abs_body_strikes_landed_avg'],
  [u'fighter_inf_body_strikes_attempts_avg',
   u'opponent_inf_body_strikes_attempts_avg'],
  [u'fighter_abs_body_strikes_attempts_avg',
  u'opponent_abs_body_strikes_attempts_avg'],
 [u'fighter_inf_leg_strikes_landed_avg',
  u'opponent_inf_leg_strikes_landed_avg'],
  [u'fighter_abs_leg_strikes_landed_avg',
  u'opponent_abs_leg_strikes_landed_avg'],
  [u'fighter_inf_leg_strikes_attempts_avg',
   u'opponent_inf_leg_strikes_attempts_avg'],
  [u'fighter_abs_leg_strikes_attempts_avg',
  u'opponent_abs_leg_strikes_attempts_avg'],
 [u'fighter_inf_distance_strikes_landed_avg',
  u'opponent_inf_distance_strikes_landed_avg'],
  [u'fighter_abs_distance_strikes_landed_avg',
  u'opponent_abs_distance_strikes_landed_avg'],
  [u'fighter_inf_distance_strikes_attempts_avg',
   u'opponent_inf_distance_strikes_attempts_avg'],
  [u'fighter_abs_distance_strikes_attempts_avg',
  u'opponent_abs_distance_strikes_attempts_avg'],
 [u'fighter_inf_ground_strikes_landed_avg',
  u'opponent_inf_ground_strikes_landed_avg'],
  [u'fighter_abs_ground_strikes_landed_avg',
  u'opponent_abs_ground_strikes_landed_avg'],
  [u'fighter_inf_ground_strikes_attempts_avg',
   u'opponent_inf_ground_strikes_attempts_avg'],
  [u'fighter_abs_ground_strikes_attempts_avg',
  u'opponent_abs_ground_strikes_attempts_avg']]

In [36]:
len(mlp_add_backs_refined)

37

In [37]:
#this adds subsets of size 1 from mlp_add_backs to mlp_base_set
mlp=MLPClassifier(max_iter=3000,alpha=.001,solver='sgd',random_state=3)
yo=ufc_fights_clean.iloc[0:2091]['method']
mlp_data_points=[]
for i in range(len(mlp_add_backs_refined)):
    candidate=[]
    candidate.extend(mlp_base_set)
    candidate.extend(mlp_add_backs_refined[i])
    Xo=ufc_fights_clean[candidate].iloc[0:2091]
    point=[i,cross_val_score(mlp,Xo,yo,cv=5).mean()]
    mlp_data_points.append(point)
    print(point)

[0, 0.6073659449528682]
[1, 0.6389107329887881]
[2, 0.6403358458817672]
[3, 0.6475209331341609]
[4, 0.646547964492368]
[5, 0.6403335565437864]
[6, 0.6446432352924343]
[7, 0.6408085941747795]
[8, 0.6355465508261648]
[9, 0.6369693743811633]
[10, 0.6456207826101886]
[11, 0.651791693137137]
[12, 0.6417792734785919]
[13, 0.6532488567618459]
[14, 0.6307778598123888]
[15, 0.6245680305397686]
[16, 0.6193174338810575]
[17, 0.634587318212256]
[18, 0.6312300040635749]
[19, 0.6283797782776166]
[20, 0.6111639566628321]
[21, 0.6580381518174482]
[22, 0.6475209331341609]
[23, 0.6518168758549246]
[24, 0.6465433858164066]
[25, 0.6475209331341609]
[26, 0.6369922677609703]
[27, 0.6456093359202852]
[28, 0.6446569713203185]
[29, 0.6398699656026968]
[30, 0.6283660422497325]
[31, 0.6369693743811633]
[32, 0.6408108835127602]
[33, 0.6484778764100891]
[34, 0.6446363672784923]
[35, 0.6470275807993224]
[36, 0.635516789432416]


In [38]:
max([tup[1] for tup in  mlp_data_points])

0.6580381518174482

In [44]:
candidate=[]
candidate.extend(mlp_base_set)
candidate.extend(mlp_add_backs_refined[21])
Xo=ufc_fights_clean[candidate].iloc[0:2091]
cross_val_score(mlp,Xo,yo,cv=6).mean()

0.6460757279622754

In [45]:
cross_val_score(mlp,Xo,yo,cv=7).mean()

0.6460642291923296

In [46]:
cross_val_score(mlp,Xo,yo,cv=8).mean()

0.6499078704922348

In [47]:
cross_val_score(mlp,Xo,yo,cv=9).mean()

0.6632528588278181

In [48]:
cross_val_score(mlp,Xo,yo,cv=10).mean()

0.6732274031587533

In [ ]:
#it could make sense to just do one at a time... this would take the least amount of time,
#less than 15 minutes each time. Then if we reach a point where it doesn't increase, we can try 2
#by this I mean, after the first round of adding 1, find the optimal choice, add this to the base set, 
#remove from add backs, and perform again. Actually one could write a program to do this automatically
#honestly its probably been done but who cares. I'm learning
#MY GOAL IS TO REACH 68 SINCE THEN THE OVERALL PROBABILITY OF PICKING BOTH THE FIGHTER AND METHOD CORRECTLY
#IS OVER 50%. THAT IS HONESLTLY IMPRESSIVE.
#another idea. after we find we cannot improve in this way, then start removing until we increase.
#then after removing, it may be possible to increase by adding